In [1]:
import matplotlib.pyplot as plt
import pytesseract
import cv2
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'

In [2]:
tz_names = os.listdir("C:/Users/kirb2/Downloads/train_sample/tz/")
npa_names = os.listdir("C:/Users/kirb2/Downloads/train_sample/npa/")

In [3]:
all_texts = []

In [4]:
#Обрабатываем все ТЗ
for name in tz_names:
    image = cv2.imread("C:/Users/kirb2/Downloads/train_sample/tz/" + name)  #Открываем картинку
    all_texts.append(pytesseract.image_to_string(image, config = r'-l rus')) #Картинка -> текст
#То же самое для НПА
for name in npa_names:
    image = cv2.imread("C:/Users/kirb2/Downloads/train_sample/npa/" + name)
    all_texts.append(pytesseract.image_to_string(image, config = r'-l rus'))

In [5]:
vectorizer = CountVectorizer()
V = vectorizer.fit_transform(all_texts)
all_words = vectorizer.get_feature_names()

In [6]:
df = pd.DataFrame(V.toarray(), columns = all_words)
classes = np.zeros(len(V.toarray()))
classes[0:len(tz_names)] = 1
e = pd.DataFrame(classes, columns = ['class'])
df['class'] = e.values
df = shuffle(df)

In [7]:
X = df.drop(['class'], axis = 1)
y = df['class']

In [8]:
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

model = LogisticRegression()
param_grid = {'C': [0.01, 0.05, 0.1, 0.5, 0.75, 1, 1.25, 1.5, 2, 5, 10]}

In [10]:
optimizer = GridSearchCV(model, param_grid, "accuracy", cv = 3)
optimizer.fit(X_train, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.01, 0.05, 0.1, 0.5, 0.75, 1, 1.25, 1.5, 2, 5,
                               10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [11]:
#с лучшим параметром, найденным по сетке
#на обучающей выборке получен такой результат
print(optimizer.best_score_)

0.9824368189195455


In [12]:
#теперь тестируем на тестовой выборке
model = optimizer.best_estimator_
prediction = model.predict(X_test)

from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, prediction)
print(score)

0.9863481228668942
